<img align="right" src="images/tf-small.png" width="90"/>
<img align="right" src="images/etcbc.png" width="100"/>

# Introduction - (Code is Work in Progress)

The code in this NB explores the coreference features that have been made with the TF conversion program in the [tf_conversion repository](). The features that have been made on the basis of annotations in *brat* are:

* mention: a mention is any referring expression; 
* coref: coreference relation between two or more mentions; 
* mentionNote: annotator notes on a mention. 

The code is still in an experimentation phase. 

In [1]:
__author__ = 'erwich/roorda'

In [2]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
from collections import defaultdict
from glob import glob
from pprint import pprint
from operator import itemgetter, attrgetter

import pandas as pd

from tf.app import use
from tf.fabric import Fabric
from utils import *

In [4]:
A = use(
    'bhsa', version='2017',
    mod=(
        'cmerwich/participant-analysis/coreference/tf:clone,'
        'etcbc/bh-reference-system/tf:clone'
    ), 
    hoist=globals())

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/Christiaan/text-fabric-data/annotation/app-bhsa/code:
	rv1.2=#5fdf1778d51d938bfe80b37b415e36618e50190c (latest release)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/bhsa/tf/2017:
	rv1.6=#bac4a9f5a2bbdede96ba6caea45e762fe88f88c5 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/phono/tf/2017:
	r1.2 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/parallels/tf/2017:
	r1.2 (latest release)
Using data in /Users/Christiaan/github/cmerwich/participant-analysis/coreference/tf/2017:
	repo clone offline under ~/github (local github)
Using data in /Users/Christiaan/github/etcbc/bh-reference-system/tf/2017:
	repo clone offline under ~/github (local github)


## Choose a corpus or text 

It has to be a chapter or corpus that has been annotated for coreference. Annotated for coference:

1. Psalms 
2. Genesis 1
3. Numbers 8-10
4. Isaiah 42:1-25

In [352]:
F.pgn_prs.freqList()

(('P3Msg', 13593),
 ('P1Csg', 7887),
 ('P3Mpl', 7153),
 ('P2Msg', 7134),
 ('P3Fsg', 3513),
 ('P2Mpl', 2667),
 ('P1Cpl', 1645),
 ('P2Fsg', 1308),
 ('P3Fpl', 239),
 ('P2Fpl', 19))

In [5]:
# Set any Hebrew Bible Book
MY_BOOK = {'Psalms'} 

# Set any range in chapters of specified HB book
MY_CHAPTERS = set(range(1,151))

In [6]:
name_types = {'pers': 'pers',
              'mens': 'mens', 
              'gens': 'gens', 
              'topo': 'topo',
              'ppde': 'ppde',
              'pers,gens,topo': 'pers,gens,topo',
              'pers,gens': 'pers,gens',
              None: '-',
             'NA': '-',
             ' ': '-'}

rpt_dict = {'subs': 'NP',
            'prep': 'PP',
            'verb': 'VP',
            'conj': 'CP',
            'nmpr': 'PrNP',
            'art': 'Art',
            'nega': 'NegP',
            'advb': 'AdvP',
            'adjv': 'AdjP',
            'prps': 'PPrP',
            'prde': 'DPrP',
            'intj': 'InjP',
            'inrg': 'InrP',
            'prin': 'InrP'
           }

In [ ]:
DataStructure: 

Psalm X

Class: C1, '<MJM'
------------------------------------------------------------
['JWDW', '<MJM', 'JWDW', '<MJM', 'KL', 'M', '<MJM', 'JWDW', '<MJM', 'JWDW', '<MJM', 'KL', 'M']
[rpt, rpt, rpt, rpt, rpt]

------------------------------------------------------------
Class: C2, '<MJM'
------------------------------------------------------------
['JWDW', '<MJM', 'JWDW', '<MJM']


nmpr > subs > verb > suffix > ?

In [15]:
def parseTFData(node, fName):
    '''
    Helper function to parse the 
    coref, mention, and mentionNote features
    stored in TF files in a structured way. 
    '''
    
    valueStr = Fs(fName).v(node)
    if not valueStr:
        return None
    parts = valueStr.split('|')
    valueData = {}
    for part in parts:
        (ct, seqNum, wordSize, isSuffix, wordPart) = part.split(',', maxsplit=4)
        valueData.setdefault(ct, {})[seqNum] = (int(wordSize or 1), isSuffix == 's', wordPart)
    print(valueData)
    return valueData

In [16]:
s = []

for book in F.otype.s('book'):
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        book_name = T.bookName(book)
        
        c = []
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:
            
            for word in L.d(chn, 'word'):

                nametype = F.nametype.v(L.u(word, 'lex')[0])

                # load coref 
                coref = parseTFData(word, 'coref')
                
                if coref:
                    for (ct, ctData) in coref.items():
                        
                        for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():
                            if ct == 'C': #if ct == 'T':
                                print(coref)
                                #print(sorted(seqNum), (wordSize, isSuffix, wordPart))
                                #print((seqNum, (wordSize, isSuffix, wordPart)))
                                c.append(int(seqNum))
                                #c.append(wordPart)
                                cc = []
                                for C in c:
                                    cc.append(C)
            
            #print(sorted(c))

{'C': {'2': (1, False, '>JC')}}
{'C': {'2': (1, False, '>JC')}}
{'C': {'2': (1, False, 'HLK')}}
{'C': {'2': (1, False, 'HLK')}}
{'C': {'4': (2, False, '<YT RC<JM')}}
{'C': {'4': (2, False, '<YT RC<JM')}}
{'C': {'7': (2, False, 'DRK XV>JM')}}
{'C': {'7': (2, False, 'DRK XV>JM')}}
{'C': {'2': (1, False, '<MD')}}
{'C': {'2': (1, False, '<MD')}}
{'T': {'6': (2, False, 'MWCB LYJM')}}
{'C': {'2': (1, False, 'JCB')}}
{'C': {'2': (1, False, 'JCB')}}
{'C': {'3': (2, False, 'TWRT JHWH')}}
{'C': {'3': (2, False, 'TWRT JHWH')}}
{'C': {'2': (1, True, 'W')}, 'T': {'40': (1, False, 'XPY')}}
{'C': {'2': (1, True, 'W')}, 'T': {'40': (1, False, 'XPY')}}
{'C': {'3': (1, True, 'W')}}
{'C': {'3': (1, True, 'W')}}
{'C': {'2': (1, False, 'JHGH')}}
{'C': {'2': (1, False, 'JHGH')}}
{'T': {'41': (1, False, 'JWMM')}}
{'T': {'42': (1, False, 'LJLH')}}
{'C': {'2': (1, False, 'HJH')}}
{'C': {'2': (1, False, 'HJH')}}
{'C': {'9': (1, False, '<Y')}}
{'C': {'9': (1, False, '<Y')}}
{'C': {'9': (1, False, 'CTWL')}}
{'C':

{'T': {'11': (2, False, 'T>WT NPC')}}
{'C': {'2': (1, True, 'W')}}
{'C': {'2': (1, True, 'W')}}
{'T': {'14': (1, False, 'BY<')}}
{'C': {'2': (1, False, 'BRK')}}
{'C': {'2': (1, False, 'BRK')}}
{'C': {'2': (1, False, 'N>Y')}}
{'C': {'2': (1, False, 'N>Y')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'2': (1, False, 'RC<')}}
{'C': {'2': (1, False, 'RC<')}}
{'T': {'19': (2, False, 'GBH >P')}}
{'C': {'2': (1, True, 'W')}}
{'C': {'2': (1, True, 'W')}}
{'C': {'2': (1, False, 'JDRC')}}
{'C': {'2': (1, False, 'JDRC')}}
{'C': {'1': (1, False, '>LHJM')}}
{'C': {'1': (1, False, '>LHJM')}}
{'C': {'6': (2, False, 'KL&MZMWTJ')}}
{'C': {'6': (2, False, 'KL&MZMWTJ')}}
{'C': {'2': (1, True, 'W')}}
{'C': {'2': (1, True, 'W')}}
{'C': {'4': (1, False, 'JXJLW')}}
{'C': {'4': (1, False, 'JXJLW')}}
{'C': {'4': (1, False, 'DRK'), '2': (1, True, 'W')}}
{'C': {'4': (1, False, 'DRK'), '2': (1, True, 'W')}}
{'C': {'4': (1, False, 'DRK'), '2': (1, True, 'W')}}
{'T': {'141': (2, False,

{'C': {'6': (1, False, '>XZH')}}
{'C': {'1': (1, True, 'K'), '15': (1, False, 'PNJ')}}
{'C': {'1': (1, True, 'K'), '15': (1, False, 'PNJ')}}
{'C': {'1': (1, True, 'K'), '15': (1, False, 'PNJ')}}
{'C': {'6': (1, False, '>FB<H')}}
{'C': {'6': (1, False, '>FB<H')}}
{'C': {'1': (1, True, 'K')}, 'T': {'137': (1, False, 'TMWNT')}}
{'C': {'1': (1, True, 'K')}, 'T': {'137': (1, False, 'TMWNT')}}
{'T': {'404': (1, False, 'MNYX')}}
{'C': {'31': (2, False, '<BD JHWH')}}
{'C': {'31': (2, False, '<BD JHWH')}}
{'C': {'31': (1, False, 'DWD')}}
{'C': {'31': (1, False, 'DWD')}}
{'C': {'31': (1, False, 'DBR')}}
{'C': {'31': (1, False, 'DBR')}}
{'C': {'9': (1, False, 'JHWH')}}
{'C': {'9': (1, False, 'JHWH')}}
{'C': {'1': (3, False, 'DBRJ HCJRH')}}
{'C': {'1': (3, False, 'DBRJ HCJRH')}}
{'C': {'1': (1, False, 'Z>T')}}
{'C': {'1': (1, False, 'Z>T')}}
{'T': {'27': (1, False, 'JWM')}}
{'C': {'9': (1, False, 'HYJL')}}
{'C': {'9': (1, False, 'HYJL')}}
{'C': {'9': (1, False, 'JHWH')}}
{'C': {'9': (1, False, 'JH

{'C': {'19': (1, False, '<NWJM')}}
{'C': {'19': (1, False, 'JFB<W')}}
{'C': {'19': (1, False, 'JFB<W')}}
{'C': {'21': (1, False, 'JHLLW')}}
{'C': {'21': (1, False, 'JHLLW')}}
{'C': {'4': (1, False, 'JHWH')}}
{'C': {'4': (1, False, 'JHWH')}}
{'C': {'21': (1, False, 'DRCJ'), '4': (1, True, 'W')}}
{'C': {'21': (1, False, 'DRCJ'), '4': (1, True, 'W')}}
{'C': {'21': (1, False, 'DRCJ'), '4': (1, True, 'W')}}
{'C': {'20': (1, False, 'JXJ')}}
{'C': {'20': (1, False, 'JXJ')}}
{'C': {'21': (1, True, 'KM'), '20': (1, False, 'LBB')}}
{'C': {'21': (1, True, 'KM'), '20': (1, False, 'LBB')}}
{'C': {'21': (1, True, 'KM'), '20': (1, False, 'LBB')}}
{'C': {'22': (1, False, 'JZKRW')}}
{'C': {'22': (1, False, 'JZKRW')}}
{'C': {'22': (1, False, 'JCBW')}}
{'C': {'22': (1, False, 'JCBW')}}
{'C': {'4': (1, False, 'JHWH')}}
{'C': {'4': (1, False, 'JHWH')}}
{'C': {'22': (3, False, 'KL&>PSJ&>RY')}}
{'C': {'22': (3, False, 'KL&>PSJ&>RY')}}
{'C': {'23': (1, False, 'JCTXWW')}}
{'C': {'23': (1, False, 'JCTXWW')}}
{'

{'C': {'5': (2, False, 'QWL&JHWH')}}
{'C': {'5': (1, False, 'XYB')}}
{'C': {'5': (1, False, 'XYB')}}
{'T': {'48': (2, False, 'LHBWT >C')}}
{'C': {'5': (2, False, 'QWL JHWH')}}
{'C': {'5': (2, False, 'QWL JHWH')}}
{'C': {'5': (1, False, 'JXJL')}}
{'C': {'5': (1, False, 'JXJL')}}
{'T': {'50': (1, False, 'MDBR')}}
{'C': {'4': (1, False, 'JXJL')}}
{'C': {'4': (1, False, 'JXJL')}}
{'C': {'4': (1, False, 'JHWH')}}
{'C': {'4': (1, False, 'JHWH')}}
{'T': {'52': (2, False, 'MDBR QDC')}}
{'C': {'5': (2, False, 'QWL JHWH')}}
{'C': {'5': (2, False, 'QWL JHWH')}}
{'C': {'5': (1, False, 'JXWLL')}}
{'C': {'5': (1, False, 'JXWLL')}}
{'T': {'54': (1, False, '>JLWT')}}
{'C': {'5': (1, False, 'JXFP')}}
{'C': {'5': (1, False, 'JXFP')}}
{'T': {'56': (1, False, 'J<RWT')}}
{'T': {'57': (1, False, 'HJKL')}, 'C': {'5': (1, True, 'W')}}
{'T': {'57': (1, False, 'HJKL')}, 'C': {'5': (1, True, 'W')}}
{'C': {'6': (1, False, 'KL'), '5': (1, True, 'W')}}
{'C': {'6': (1, False, 'KL'), '5': (1, True, 'W')}}
{'C': {'6':

...

{'C': {'14': (2, False, 'BJT JFR>L')}}
{'C': {'1': (1, False, 'JBRK')}}
{'C': {'1': (1, False, 'JBRK')}}
{'C': {'8': (2, False, 'BJT >HRN')}}
{'C': {'8': (2, False, 'BJT >HRN')}}
{'C': {'1': (1, False, 'JBRK')}}
{'C': {'1': (1, False, 'JBRK')}}
{'C': {'11': (2, False, 'JR>J JHWH')}}
{'C': {'11': (2, False, 'JR>J JHWH')}}
{'T': {'89': (1, False, 'QVNJM')}}
{'T': {'90': (1, False, 'GDLJM')}}
{'C': {'1': (1, False, 'JSP')}}
{'C': {'1': (1, False, 'JSP')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'11': (1, True, 'KM')}}
{'C': {'11': (1, True, 'KM')}}
{'C': {'11': (1, True, 'KM')}}
{'C': {'11': (1, True, 'KM')}}
{'T': {'95': (1, False, 'BNJ')}, 'C': {'11': (1, True, 'KM')}}
{'T': {'95': (1, False, 'BNJ')}, 'C': {'11': (1, True, 'KM')}}
{'C': {'11': (1, False, 'BRWKJM')}}
{'C': {'11': (1, False, 'BRWKJM')}}
{'C': {'11': (1, True, 'M')}}
{'C': {'11': (1, True, 'M')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'1': (1, False, 'JHWH')}}
{'C': {'1': (4, False, '<FH C

{'C': {'54': (1, False, 'CMRTJ')}}
{'C': {'54': (1, False, 'CMRTJ')}}
{'C': {'37': (1, False, '>TH')}}
{'C': {'37': (1, False, '>TH')}}
{'C': {'37': (1, False, 'MVJB')}}
{'C': {'37': (1, False, 'MVJB')}}
{'C': {'37': (1, False, 'LMD'), '54': (1, True, 'NJ')}}
{'C': {'37': (1, False, 'LMD'), '54': (1, True, 'NJ')}}
{'C': {'37': (1, False, 'LMD'), '54': (1, True, 'NJ')}}
{'C': {'37': (1, True, 'K'), '11': (1, False, 'XQJ')}}
{'C': {'37': (1, True, 'K'), '11': (1, False, 'XQJ')}}
{'C': {'37': (1, True, 'K'), '11': (1, False, 'XQJ')}}
{'C': {'36': (1, False, 'VPLW')}}
{'C': {'36': (1, False, 'VPLW')}}
{'C': {'54': (1, True, 'J')}}
{'C': {'54': (1, True, 'J')}}
{'T': {'451': (1, False, 'CQR')}}
{'C': {'36': (1, False, 'ZDJM')}}
{'C': {'36': (1, False, 'ZDJM')}}
{'C': {'54': (1, False, '>NJ')}}
{'C': {'54': (1, False, '>NJ')}}
{'C': {'53': (2, False, 'KL&LB')}}
{'C': {'53': (2, False, 'KL&LB')}}
{'C': {'54': (1, False, '>YR')}}
{'C': {'54': (1, False, '>YR')}}
{'C': {'37': (1, True, 'K'), '2

{'C': {'3': (2, False, 'LCWN RMJH')}}
{'C': {'3': (2, False, 'LCWN RMJH')}}
{'C': {'2': (1, False, 'JTN')}}
{'C': {'2': (1, False, 'JTN')}}
{'C': {'3': (1, True, 'K')}}
{'C': {'3': (1, True, 'K')}}
{'C': {'2': (1, False, 'JSJP')}}
{'C': {'2': (1, False, 'JSJP')}}
{'C': {'3': (1, True, 'K')}}
{'C': {'3': (1, True, 'K')}}
{'C': {'3': (2, False, 'LCWN RMJH')}}
{'C': {'3': (2, False, 'LCWN RMJH')}}
{'T': {'19': (2, False, 'XYJ GBWR')}}
{'T': {'20': (1, False, 'CNWNJM')}}
{'T': {'21': (2, False, 'GXLJ RTMJM')}}
{'C': {'1': (1, True, 'J')}}
{'C': {'1': (1, True, 'J')}}
{'C': {'1': (1, False, 'GRTJ')}}
{'C': {'1': (1, False, 'GRTJ')}}
{'T': {'24': (1, False, 'MCK')}}
{'C': {'1': (1, False, 'CKNTJ')}}
{'C': {'1': (1, False, 'CKNTJ')}}
{'T': {'26': (2, False, '>HLJ QDR')}}
{'C': {'4': (1, False, 'CKNH')}}
{'C': {'4': (1, False, 'CKNH')}}
{'C': {'4': (1, True, 'H')}}
{'C': {'4': (1, True, 'H')}}
{'C': {'1': (1, True, 'J'), '4': (1, False, 'NPC')}}
{'C': {'1': (1, True, 'J'), '4': (1, False, 'NPC

{'C': {'8': (1, False, 'ZKR')}}
{'C': {'8': (1, False, 'ZKR')}}
{'C': {'8': (1, False, 'JHWH')}}
{'C': {'8': (1, False, 'JHWH')}}
{'C': {'9': (2, False, 'BNJ >DWM')}}
{'C': {'9': (2, False, 'BNJ >DWM')}}
{'T': {'46': (2, False, 'JWM JRWCLM')}}
{'C': {'9': (1, False, '>MRJM')}}
{'C': {'9': (1, False, '>MRJM')}}
{'C': {'10': (1, False, '<RW')}}
{'C': {'10': (1, False, '<RW')}}
{'C': {'10': (1, False, '<RW')}}
{'C': {'10': (1, False, '<RW')}}
{'T': {'50': (1, False, 'JSWD')}}
{'C': {'4': (1, True, 'H')}}
{'C': {'4': (1, True, 'H')}}
{'C': {'11': (2, False, 'BT&BBL')}}
{'C': {'11': (2, False, 'BT&BBL')}}
{'C': {'11': (1, False, 'CDWDH')}}
{'C': {'11': (1, False, 'CDWDH')}}
{'C': {'12': (1, False, '>CRJ')}}
{'C': {'12': (1, False, '>CRJ')}}
{'C': {'12': (1, False, 'JCLM')}}
{'C': {'12': (1, False, 'JCLM')}}
{'C': {'11': (1, True, 'K')}}
{'C': {'11': (1, True, 'K')}}
{'T': {'57': (1, False, 'GMWL')}, 'C': {'11': (1, True, 'K')}}
{'T': {'57': (1, False, 'GMWL')}, 'C': {'11': (1, True, 'K')}}


{'C': {'5': (1, False, 'RWX'), '4': (1, True, 'W')}}
{'C': {'5': (1, False, 'RWX'), '4': (1, True, 'W')}}
{'C': {'5': (1, False, 'RWX'), '4': (1, True, 'W')}}
{'C': {'4': (1, False, 'JCB')}}
{'C': {'4': (1, False, 'JCB')}}
{'T': {'25': (1, False, '>DMT')}, 'C': {'4': (1, True, 'W')}}
{'T': {'25': (1, False, '>DMT')}, 'C': {'4': (1, True, 'W')}}
{'C': {'6': (1, False, 'JWM')}}
{'C': {'6': (1, False, 'JWM')}}
{'C': {'6': (1, False, 'W>')}}
{'C': {'6': (1, False, 'W>')}}
{'C': {'4': (1, False, '>BDW')}}
{'C': {'4': (1, False, '>BDW')}}
{'T': {'30': (1, False, '<CTNTJ')}, 'C': {'4': (1, True, 'W')}}
{'T': {'30': (1, False, '<CTNTJ')}, 'C': {'4': (1, True, 'W')}}
{'C': {'7': (1, False, '>CRJ')}}
{'C': {'7': (1, False, '>CRJ')}}
{'C': {'9': (2, False, '>L J<QB')}}
{'C': {'9': (2, False, '>L J<QB')}}
{'T': {'34': (1, False, '<ZR')}, 'C': {'7': (1, True, 'W')}}
{'T': {'34': (1, False, '<ZR')}, 'C': {'7': (1, True, 'W')}}
{'T': {'36': (1, False, 'FBR')}, 'C': {'7': (1, True, 'W')}}
{'T': {'36':

In [123]:
def Analyse(book, chapter):
    valueData = ParseTFData(node, fName)
    rpt = ReconstructPtype(valueData) # returns reconstructed phrase types 
    class_info, sing_info = RetrievCorefInfo(valueData)
    coref_lists, singleton_list = RetrieveAnn(valueData)
    participants = IdentifyClass(coref_lists)
    PrintAnalysis(participants, coref_lists, singleton_list)
    
def Analyse('Psalms', 8, 10)

# Prepare all coreference and singleton information

In [11]:
coref_tuples = []
singleton_tuples = []

for book in F.otype.s('book'):
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        book_name = T.bookName(book)
        
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:
           
            for word in L.d(chn, 'word'):

                nametype = F.nametype.v(L.u(word, 'lex')[0])

                # load all needed features 
                coref = parseTFData(word, 'coref')

                if not coref and nametype:
                    lex_text = T.text(word, fmt='lex-trans-plain')
                    #error(f'NO COREF for NAMETYPE {nametype}: {lex_text} ({word})')

                # extract coref feature 
                if coref:
                    #print(coref)
                    vt = F.vt.v(word)
                    person = F.ps.v(word)
                    st = F.st.v(word)
                    lex_text = T.text(word, fmt='lex-trans-plain')                       
                    bo, ch, vs = T.sectionFromNode(word)
                    text_pos = str(ch)+'.'+str(vs)
                    pdpos = F.pdp.v(word)
                    phrase_type = F.typ.v(L.u(word, 'phrase')[0])
                    det = F.det.v(L.u(word, 'phrase')[0])

                    for (ct, ctData) in coref.items():
                        dest = singleton_tuples if ct == 'T' else coref_tuples
                        for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():

                            cls = f'{ct}{seqNum}'
                            text = wordPart
                            ne = '-' if isSuffix else name_types[nametype]
                            pgn = converse_pgn_suffix(wordPart) if isSuffix else converse_pgn(F, word)
                            pdpos = 'suffix' if isSuffix else F.pdp.v(word)
                            rpt = 'suffix' if isSuffix else rpt_dict[F.pdp.v(word)]
                            dt = 'det' if isSuffix else det
                            dest.append((
                                    word,
                                    cls,
                                    text,
                                    ne,
                                    pgn,
                                    text_pos,
                                    pdpos,
                                    st,
                                    rpt,
                                    vt,
                                    dt,
                            ))                           
#pprint(singleton_tuples)
#pprint(coref_tuples)

# View on the coref after parsing

# Make coref sets

Here all coref sets are made and the number of unique coreference relations, or classes, are counted. 


In [12]:
info('Counting ... \n')

my_mention_counter = 0
my_mention_note_counter = 0
my_appo_counter = 0
my_entity_counter = 0

my_coref_sets = []
my_classes_count = [] 
sets_dict = defaultdict()

for book in F.otype.s('book'):
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        book_name = T.bookName(book)
        #bo, ch, vs = T.sectionFromNode(chn)
        
        corefSet = set()
        sets_dict[chapter] = corefSet

        if book_name in MY_BOOK and chapter in MY_CHAPTERS:

            for phr_atom in L.d(chn, 'phrase_atom'):
                relas = F.rela.v(phr_atom)
                if relas == 'Appo':
                    my_appo_counter +=1

            for word in L.d(chn, 'word'):
                mentions = F.mention.v(word)
                mention_notes = F.mentionNote.v(word)
                nametypes = F.nametype.v(L.u(word, 'lex')[0])

                if nametypes:
                    my_entity_counter += 1

                if mention_notes: 
                    my_mention_note_counter +=1

                if mentions:
                    my_mention_counter +=1

                # load all needed features 
                coref = parseTFData(word, 'coref')

                # extract coref feature 
                if coref:
                    for (ct, ctData) in coref.items():
                        if ct == 'T':
                            continue
                        for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():
                            cls = f'{ct}{seqNum}'
                            corefSet.add(cls)

        my_coref_sets.append(corefSet)

#print(my_coref_sets)

for items in my_coref_sets:

    lens = len(items)
    my_classes_count.append(lens)

print(
    'There are {} participant classes in your corpus: \n\n {}\n{}'.format(
    sum(my_classes_count),MY_BOOK,MY_CHAPTERS
    ))

#print(sets_dict)

 1m 04s Counting ... 

There are 2001 participant classes in your corpus: 

 {'Psalms'}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


# Count all coference features for all coreference-annotated texts


In [13]:
info('Counting all annotated mentions, classes and mention notes...\n')

total_mentions = 0
total_mention_notes = 0
total_appositions = 0
total_entities = 0
coref_counter_set = []
total_classes = []

for book in F.otype.s('book'):
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        book_name = T.bookName(book)
        corefSets = set()
        
        for phr_atom in L.d(chn, 'phrase_atom'):
            relas = F.rela.v(phr_atom)
            if relas == 'Appo':
                total_appositions +=1

        for word in L.d(chn, 'word'):
            mentions = F.mention.v(word)
            mention_notes = F.mentionNote.v(word)
            nametypes = F.nametype.v(L.u(word, 'lex')[0])
            
            if nametypes:
                total_entities += 1

            if mention_notes: 
                total_mention_notes +=1

            if mentions:
                total_mentions +=1
            
            # load all needed features 
            corefs = parseTFData(word, 'coref')
            
            # extract coref feature 
            if corefs:
                for (ct, ctData) in corefs.items():
                    if ct == 'T':
                        continue
                    for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():
                        cls = f'{ct}{seqNum}'
                        corefSets.add(cls)
                             
                                
                                
        coref_counter_set.append(corefSets)
for cl in coref_counter_set:
    lengths = len(cl)
    total_classes.append(lengths)
info('Counting is done!')

 1m 09s Counting all annotated mentions, classes and mention notes...

 1m 19s Counting is done!


# Print the counts

In [14]:
percent_mentions = round((my_mention_counter/total_mentions)*100,1)
percent_participant_classes = round((sum(my_classes_count)/sum(total_classes))*100,1)
percent_appositions = round((my_appo_counter/total_appositions)*100,1)
percent_entities = round((my_entity_counter/total_entities)*100,1)
percent_mention_notes = round((my_mention_note_counter/total_mention_notes)*100,1)

print('Your corpus and all coreference-annotated corpora consist of: \n\n \
      {:<5}/{} Mentions\n \
      {:<5}/{} Participant classes\n \
      {:<5}/{} Appositions\n \
      {:<5}/{} Named Entities\n \
      {:<5}/{} Annotator notes on those mentions\n\n'.format(
    my_mention_counter, 
    total_mentions, 
    sum(my_classes_count), 
    sum(total_classes), 
    my_appo_counter, 
    total_appositions, 
    my_entity_counter, 
    total_entities, 
    my_mention_note_counter, 
    total_mention_notes
    ))

print('You are studying: \n\n \
      {}% of all Mentions\n \
      {}% of all Participant classes\n \
      {}% of all Appositions\n \
      {}% of all Named Entities\n \
      {}% of all Annotator notes'.format(
    percent_mentions, 
    percent_participant_classes, 
    percent_appositions, 
    percent_entities,
    percent_mention_notes
    ))

Your corpus and all coreference-annotated corpora consist of: 

       15195/26562 Mentions
       2001 /3787 Participant classes
       131  /5887 Appositions
       1330 /35569 Named Entities
       712  /3733 Annotator notes on those mentions


You are studying: 

       57.2% of all Mentions
       52.8% of all Participant classes
       2.2% of all Appositions
       3.7% of all Named Entities
       19.1% of all Annotator notes


# Print singletons 

A singleton is one mention that forms it's own class. It does not belong to a class with other mentions. 

--TO DO: most of the information in the table below is right, except for the phrase types (`phr_type`). Mentions can contain multiple words within a phrase, but does not necessarily have to be so. The `phr_type` that is printed now is the `phr_type` in which the mention was originally contained. To illustrate: T1 is a named entity, not a prepositional phrase. 

In [123]:
# print singletons 
# !!! check exceptions here.!!!

def print_singletons():
    
    print('{:<5}{:<6}{:>15}{:>14}{:>11}{:>12}{:>10}{:>10}{:>13}{:>9}{:>10}\n{}'.format(
    'node', 'sing', 'mention', 'NE', 'pgn', 'text', 'pdpos', 'st', 'rphr_type', 'vt', 'det', '-'*117))
    
    #sort on word node, canonical order
    for s in sorted (singleton_tuples, key=itemgetter(0)):
        print('{:<5} {:<6} {:>15} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(*s))

print_singletons()

node sing          mention            NE        pgn        text     pdpos        st    rphr_type       vt       det
---------------------------------------------------------------------------------------------------------------------
310667 T6           MWCB LYJM          -        msg        1.1       subs          c         NP         NA        und
310676 T40                XPY          -        msg        1.2       subs          a         NP         NA        det
310681 T41               JWMM          -                   1.2       advb         NA       AdvP         NA         NA
310683 T42               LJLH          -        msg        1.2       advb          a       AdvP         NA         NA
310690 T43           PLGJ MJM          -        mpl        1.3       subs          c         NP         NA        und
310693 T17                PRJ          -        msg        1.3       subs          a         NP         NA        det
310696 T44                 <T          -        usg       

321092 T101               NDR          -        mpl      66.13       subs          a         NP         NA        det
321101 T108                YR          -        msg      66.14       subs          a         NP         NA        det
321103 T110         <LWT MXJM          -        fpl      66.15       subs          c         NP         NA        und
321108 T113        QVRT >JLJM          -        fsg      66.15       subs          c         NP         NA        und
321111 T115               BQR          -        usg      66.15       subs          a         NP         NA        und
321113 T116            <TWDJM          -        mpl      66.15       subs          a         NP         NA        und
321119 T120     KL&JR>J >LHJM          -        msg      66.16       subs          c         NP         NA        und
321125 T123               NPC          -        fsg      66.16       subs          a         NP         NA        det
321130 T128              RWMM          -        msg     

330264 T224                 P          -        msg     109.30       subs          a         NP         NA        det
330267 T225          TWK RBJM          -        msg     109.30       subs          c         NP         NA        und
330277 T231         MCPVJ NPC          -                109.31       prep         NA         PP         NA        det
330281 T1                 DWD       pers        msg      110.1       nmpr          a       PrNP         NA        det
330282 T2               MZMWR          -        msg      110.1       subs          a         NP         NA        und
330286 T4                   J          -      p1usg      110.1     suffix          a     suffix         NA        det
330289 T7                JMJN          -        fsg      110.1       subs          a         NP         NA        det
330292 T10               >JBJ          -       umpl      110.1       subs          a         NP       ptca        det
330293 T12                HDM          -        msg     

# Print all participant classes

--TO DO: as with the singletons most of the information in the table below is right, except for the phrase types (`phr_type`). Mentions can contain multiple words within a phrase, but does not necessarily have to be so. The `phr_type` that is printed now is the `phr_type` in which the mention was originally contained. The fix is to isolate the head of the phrase.

In [124]:
#print participant classes
# !!! check exceptions here.!!!

def print_participant_classes():
    
    print('{:<5}{:<6}{:>17}{:>13}{:>10}{:>12}{:>10}{:>10}{:>13}{:>9}{:>10}\n{}'.format(
    'node', 'class', 'mention', 'NE', 'pgn', 'text', 'pdpos', 'st', 'rphr_type', 'vt', 'det', '-'*117))
    
    #sort on word node, canonical order
    for c in sorted(coref_tuples, key=itemgetter(0)):
        print('{:<5} {:<6} {:>15} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(*c))
        #print((c[3]),type(c[3]))
print_participant_classes()

node class           mention           NE       pgn        text     pdpos        st    rphr_type       vt       det
---------------------------------------------------------------------------------------------------------------------
310652 C2                 >JC          -        msg        1.1       subs          a         NP         NA        det
310655 C2                 HLK          -      p3msg        1.1       verb         NA         VP       perf         NA
310657 C4           <YT RC<JM          -        fsg        1.1       subs          c         NP         NA        und
310661 C7           DRK XV>JM          -        usg        1.1       subs          c         NP         NA        und
310664 C2                 <MD          -      p3msg        1.1       verb         NA         VP       perf         NA
310670 C2                 JCB          -      p3msg        1.1       verb         NA         VP       perf         NA
310674 C3           TWRT JHWH          -        fsg       

313954 C1                   J          -      p1usg       23.6     suffix          a     suffix         NA        det
313958 C2            BJT&JHWH          -        msg       23.6       subs          c         NP         NA        det
313967 C2                JHWH       pers        msg       24.1       nmpr          a       PrNP         NA        det
313969 C1                 >RY          -        usg       24.1       subs          a         NP         NA        det
313971 C1                   H          -      p3fsg       24.1     suffix          a     suffix         NA        det
313972 C1                 TBL          -        fsg       24.1       subs          a         NP         NA        und
313975 C1                   H          -      p3fsg       24.1     suffix         NA     suffix         NA        det
313977 C2                 HW>       ppde      p3msg       24.2       prps         NA       PPrP         NA        det
313980 C1                   H          -      p3fsg     

317262 C8                HPKT          -      p2msg       41.4       verb         NA         VP       perf         NA
317264 C1                   W          -      p3msg       41.4     suffix          a     suffix         NA        det
317265 C7                 >NJ          -      p1usg       41.5       prps         NA       PPrP         NA        det
317266 C7               >MRTJ          -      p1usg       41.5       verb         NA         VP       perf         NA
317267 C8                JHWH       pers        msg       41.5       nmpr          a       PrNP         NA        det
317268 C7                  NJ          -      p1usg       41.5     suffix         NA     suffix       impv        det
317268 C8                  XN          -      p2msg       41.5       verb         NA         VP       impv         NA
317269 C8                RP>H          -      p2msg       41.5       verb         NA         VP       impv         NA
317270 C7                   J          -      p1usg     

320349 C5                  NW          -      p1upl      60.12     suffix         NA     suffix       perf        det
320349 C3                ZNXT          -      p2msg      60.12       verb         NA         VP       perf         NA
320352 C3                 TY>          -      p2msg      60.12       verb         NA         VP       impf         NA
320353 C3               >LHJM          -        mpl      60.12       subs          a         NP         NA        und
320355 C5                  NW          -      p1upl      60.12     suffix          a     suffix         NA        det
320356 C3                 HBH          -      p2msg      60.13       verb         NA         VP       impv         NA
320357 C5                  NW          -      p1upl      60.13     suffix         NA     suffix         NA        det
320360 C11                 YR          -        msg      60.13       subs          a         NP         NA        und
320366 C3               >LHJM          -        mpl     

322924 C12         R>CJ LWJTN          -        mpl      74.14       subs          c         NP         NA        und
322926 C18               TTNN          -      p2msg      74.14       verb         NA         VP       impf         NA
322926 C12                  W          -      p3msg      74.14     suffix         NA     suffix       impf        det
322932 C18                >TH          -      p2msg      74.15       prps         NA       PPrP         NA        det
322933 C18               BQ<T          -      p2msg      74.15       verb         NA         VP       perf         NA
322937 C18                >TH          -      p2msg      74.15       prps         NA       PPrP         NA        det
322938 C18              HWBCT          -      p2msg      74.15       verb         NA         VP       perf         NA
322941 C18                  K          -      p2msg      74.16     suffix         NA     suffix         NA        det
322944 C18                  K          -      p2msg     

325202 C3                  <M          -        msg       85.7       subs          a         NP         NA        det
325202 C9                   K          -      p2msg       85.7     suffix          a     suffix         NA        det
325203 C3               JFMXW          -      p3mpl       85.7       verb         NA         VP       impf         NA
325204 C9                   K          -      p2msg       85.7     suffix         NA     suffix         NA        det
325205 C9                 HR>          -      p2msg       85.8       verb         NA         VP       impv         NA
325205 C2                  NW          -      p1upl       85.8     suffix         NA     suffix       impv        det
325206 C9                JHWH       pers        msg       85.8       nmpr          a       PrNP         NA        det
325207 C9                   K          -      p2msg       85.8     suffix          a     suffix         NA        det
325209 C4                 JC<          -        msg     

327797 C9                BWDD          -       umsg      102.8       verb          a         VP       ptca         NA
327803 C9                  NJ          -      p1usg      102.9     suffix         NA     suffix       perf        det
327803 C7                XRPW          -      p3upl      102.9       verb         NA         VP       perf         NA
327804 C7                >WJB          -       umpl      102.9       subs          a         NP       ptca        det
327804 C9                   J          -      p1usg      102.9     suffix          a     suffix       ptca        det
327805 C9                   J          -      p1usg      102.9     suffix          a     suffix       ptca        det
327805 C8               MHWLL          -       umpl      102.9       subs          a         NP       ptca        det
327806 C9                   J          -      p1usg      102.9     suffix         NA     suffix         NA        det
327807 C8               NCB<W          -      p3upl     

331607 C2                PQDJ          -        mpl     119.40       subs          a         NP         NA        det
331609 C37                  K          -      p2msg     119.40     suffix          a     suffix         NA        det
331609 C46               YDQT          -        fsg     119.40       subs          a         NP         NA        det
331610 C54                 NJ          -      p1usg     119.40     suffix         NA     suffix       impv        det
331610 C37                 XJ          -      p2msg     119.40       verb         NA         VP       impv         NA
331612 C19                JB>          -      p3mpl     119.41       verb         NA         VP       impf         NA
331612 C54                 NJ          -      p1usg     119.41     suffix         NA     suffix       impf        det
331613 C37                  K          -      p2msg     119.41     suffix          a     suffix         NA        det
331613 C19                XSD          -        mpl     

335015 C3                 >LK          -      p1usg      143.8       verb         NA         VP       impf         NA
335017 C12                  K          -      p2msg      143.8     suffix         NA     suffix         NA        det
335018 C3               NF>TJ          -      p1usg      143.8       verb         NA         VP       perf         NA
335019 C3                   J          -      p1usg      143.8     suffix          a     suffix         NA        det
335019 C5                 NPC          -        fsg      143.8       subs          a         NP         NA        det
335020 C12               HYJL          -      p2msg      143.9       verb         NA         VP       impv         NA
335020 C3                  NJ          -      p1usg      143.9     suffix         NA     suffix       impv        det
335022 C8                 >JB          -       umpl      143.9       subs          a         NP       ptca        det
335022 C3                   J          -      p1usg     

### 1. Print all coreference chains
* with all words
* with all POS

In [311]:
def CountCTInfo(MY_BOOK, MY_CHAPTERS):

    class_pronoun_dict = defaultdict(lambda: defaultdict(int))
    sing_pronoun_dict = defaultdict(lambda: defaultdict(int))
    class_pron_count = collections.Counter()
    sing_pron_count = collections.Counter()
    
    class_pos_dict = collections.Counter()
    sing_pos_dict = collections.Counter()

    class_rpt_dict = collections.Counter()
    sing_rpt_dict = collections.Counter()

    for book in F.otype.s('book'):
        for chn in L.d(book, 'chapter'):
            chapter = F.chapter.v(chn)
            book_name = T.bookName(book)

            if book_name in MY_BOOK and chapter in MY_CHAPTERS:

                for word in L.d(chn, 'word'):

                    nametype = F.nametype.v(L.u(word, 'lex')[0])

                    # load all needed features 
                    coref = parseTFData(word, 'coref')

                    # extract coref feature 
                    if coref:

                        for (ct, ctData) in coref.items():
                            for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():
                                if ct == 'T':
                                    pos = 'suffix' if isSuffix else F.pdp.v(word)
                                    if pos in {'verb', 'prps'}:
                                        sing_pronoun_dict[pos][converse_pgn(F, word)] += 1
                                        sing_pron_count[converse_pgn(F, word)] += 1
                                    elif pos == 'suffix':
                                        sing_pronoun_dict[pos][suffix_dict[wordPart][0]] += 1
                                        sing_pron_count[suffix_dict[wordPart][0]] += 1
                                    if pos:
                                        sing_pos_dict[pos] += 1
                                    
                                    rpt = 'suffix' if isSuffix else rpt_dict[F.pdp.v(word)]
                                    if rpt:
                                        sing_rpt_dict[rpt] += 1  
                                else:
                                    pos = 'suffix' if isSuffix else F.pdp.v(word)
                                    if pos in {'verb', 'prps'}:
                                        class_pronoun_dict[pos][converse_pgn(F, word)] += 1
                                        class_pron_count[converse_pgn(F, word)] += 1
                                    elif pos == 'suffix':
                                        class_pronoun_dict[pos][suffix_dict[wordPart][0]] += 1
                                        class_pron_count[suffix_dict[wordPart][0]] += 1 
                                    if pos:
                                        class_pos_dict[pos] += 1
                                    rpt = 'suffix' if isSuffix else rpt_dict[F.pdp.v(word)]
                                    if rpt:
                                        class_rpt_dict[rpt] += 1
    #pprint(sing_pos_dict)
    return class_pronoun_dict, class_pron_count, class_pos_dict, sing_pronoun_dict, sing_pron_count, sing_pos_dict
#CountCTInfo('Psalms', set(range(1, 151)))

In [313]:
class_pronoun_dict, class_pron_count, class_pos_dict, \
sing_pronoun_dict, sing_pron_count, sing_pos_dict = CountCTInfo('Psalms', set(range(1, 151)))

In [350]:
class_pos_df = pd.DataFrame.from_dict(class_pos_dict, 
                                      orient='index', 
                                      columns=['class']).fillna(0).astype(int)

sing_pos_df = pd.DataFrame.from_dict(sing_pos_dict, 
                                     orient='index', 
                                     columns=['sing']).fillna(0).astype(int)

# Part of speech per class and singletons 
pos_df = class_pos_df.merge(sing_pos_df, 
                            left_index = True, 
                            right_index = True)

pos_df['total'] = pos_df.sum(axis=1)
pos_df

,class,sing,total
subs,3051,4401,7452
verb,4997,98,5095
suffix,4556,35,4591
nmpr,798,164,962
prps,287,2,289
advb,3,20,23
prde,32,15,47
adjv,7,33,40
conj,7,3,10
art,2,2,4


In [348]:
class_pron_df = pd.DataFrame.from_dict(class_pron_count, 
                                       orient='index', 
                                       columns=['class']).astype(int)

sing_pron_df = pd.DataFrame.from_dict(sing_pron_count, 
                                      orient='index', 
                                      columns=['sing']).astype(int)

pron_df = class_pron_df.merge(sing_pron_df, 
                            left_index = True, 
                            right_index = True).sort_values(
                         by='class', axis=0, ascending=False)

# verb, independent personal pronouns and suffixes per class and singletons 
pron_df['total'] = pron_df.sum(axis=1)
pron_df

,class,sing,total
p1usg,2415,13,2428
p2msg,2280,16,2296
p3msg,2087,34,2121
p3mpl,1172,8,1180
p3upl,385,7,392
p3fsg,342,3,345
p1upl,332,12,344
umsg,297,8,305
p2mpl,281,10,291
umpl,95,10,105


In [335]:
class_pronpos_df = pd.DataFrame.from_dict(class_pronoun_dict, orient='index')#.astype(int) ##.fillna(0).
sing_pronpos_df = pd.DataFrame.from_dict(sing_pronoun_dict, orient='index')#.astype(int) #.fillna(0)

class_pronpos_df.sort_values(
                        by='p3msg', 
                        axis=0, 
                        ascending=False)

,p3msg,umsg,p3fsg,p3mpl,p3upl,p1upl,p1usg,p2msg,p2mpl,umpl,uuu,p2fsg,p3fpl,ufsg,ufpl
verb,1131,297.0,234,590,385.0,91,769,998,254.0,95.0,78.0,29.0,20.0,21.0,5.0
suffix,907,NaN,102,558,NaN,233,1564,1165,27.0,NaN,NaN,NaN,NaN,NaN,NaN
prps,49,NaN,6,24,NaN,8,82,117,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [351]:
sing_pronpos_df.sort_values(
                        by='p3msg', 
                        axis=0, 
                        ascending=False)

,p2msg,p1upl,p1usg,p3mpl,p3msg,p2mpl,p3fsg,ufsg,umsg,uuu,umpl,p3upl
verb,12.0,NaN,5,4,28.0,9.0,1.0,1.0,8.0,13.0,10.0,7.0
suffix,4.0,12.0,7,3,6.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
prps,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
def AreEqual(seqNum):
    res = []
    lambda x, y: x == y

    l = next((x for x in res if areTheSame(seqNum, x[0])), [])
    if l == []:
        res.append(l)
    l.append(w)
    
    return res 

def isEqual(data):
    
    l = [list(g) for k, g in groupby(data)]

    return l

def getC(coref):
    l = []
    for (ct, ctData) in coref.items():
        for (seqNum, (wordSize, isSuffix, wordPart)) in ctData.items():
            if ct == 'C': 
                l.append(seqNum)
    return l
                #return seqNum, wordSize, isSuffix, wordPart

def GetChains(MY_BOOK, MY_CHAPTERS):

    for book in F.otype.s('book'):
        for chn in L.d(book, 'chapter'):
            chapter = F.chapter.v(chn)
            book_name = T.bookName(book)
            ch_coref_set = set()
            ch_list = []
            
            if book_name in MY_BOOK and chapter in MY_CHAPTERS:

                for word in L.d(chn, 'word'):
                    # load all needed features 
                    coref = parseTFData(word, 'coref')
                    
                    # extract coref feature 
                    if coref:
                        l = getC(coref)
                        #ch_list.append(seqNum)
                        nl = isEqual(sorted(l))
                        print(nl)

                #print(sorted(ch_coref_set))

GetChains('Psalms', set(range(1, 3)))

[['2']]
[['2']]
[['4']]
[['7']]
[['2']]
[]
[['2']]
[['3']]
[['2']]
[['3']]
[['2']]
[]
[]
[['2']]
[['9']]
[['9']]
[]
[['2']]
[['2']]
[['2']]
[]
[['2']]
[['2']]
[]
[['2']]
[['2']]
[['4']]
[['6']]
[['5'], ['6']]
[['5']]
[['4']]
[['4']]
[]
[['7']]
[['8']]
[['1']]
[['1']]
[['8']]
[['4']]
[['4']]
[['1']]
[['1']]
[['2']]
[['2']]
[]
[['5']]
[['5']]
[['5']]
[['5']]
[['8']]
[['6'], ['8']]
[['9']]
[['10']]
[['9']]
[['9']]
[['10']]
[['8']]
[]
[['8']]
[['8']]
[['8']]
[['5']]
[['8']]
[['5']]
[['8']]
[['8']]
[['5'], ['8']]
[['8']]
[['8']]
[['4'], ['8']]
[['3']]
[['3']]
[['8']]
[['4']]
[['8']]
[['8']]
[['4']]
[['6'], ['8']]
[['4']]
[['8']]
[]
[['4'], ['8']]
[['4']]
[['8']]
[['8']]
[['1']]
[['4']]
[['4']]
[]
[['1'], ['4']]
[]
[]
[['1'], ['4']]
[['5']]
[['5']]
[['5']]
[['5']]
[['5']]
[['8']]
[]
[['5']]
[]
[['5']]
[['6']]
[['6']]
[['5']]
[]
[['6']]
[['6']]
[['7']]
[['7']]
[['6']]
